In [ ]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

from collections import defaultdict
from collections import Counter

import keras
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import (LSTM, 
                          Embedding, 
                          BatchNormalization,
                          Dense, 
                          TimeDistributed, 
                          Dropout, 
                          Bidirectional,
                          Flatten, 
                          GlobalMaxPool1D)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam

from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    classification_report,
    accuracy_score
)

In [ ]:
primary_blue = "#496595"
primary_blue2 = "#85a1c1"
primary_blue3 = "#3f4d63"
primary_grey = "#c6ccd8"
primary_black = "#202022"
primary_bgcolor = "#f4f0ea"

primary_green = px.colors.qualitative.Plotly[2]

In [ ]:
df = pd.read_csv("song_data.csv")

df = df.dropna(how="any", axis=1)


In [ ]:
df.columns

Index(['Unnamed: 0', 'artist', 'song', 'link', 'text', 'explicit_label'], dtype='object')

In [ ]:
df[['text','explicit']] = df[['text', 'explicit_label']]
df = df.drop(['Unnamed: 0', 'artist', 'song', 'link', 'explicit_label'], axis=1)

In [ ]:
df.head()

,text,explicit
0,"Look at her face, it's a wonderful face \nAnd...",no match
1,"Take it easy with me, please \nTouch me gentl...",False
2,I'll never know why I had to go \nWhy I had t...,False
3,Making somebody happy is a question of give an...,no match
4,Making somebody happy is a question of give an...,False


In [ ]:
df = df.loc[df['explicit'] != 'no match']
re_drop = re.compile(r'\n')        
df[['text']] = df[['text']].applymap(lambda x:re_drop.sub(' ',x))


In [ ]:
df_exp = pd.read_csv("explicit.csv")

df_exp = df_exp[['artist','song','text','explicit_label']]
re_drop = re.compile(r'\n')        
df_exp[['text']] = df_exp[['text']].applymap(lambda x:re_drop.sub(' ',x))

In [ ]:
df_exp = df_exp.drop(['artist','song'], axis=1)

In [ ]:
df_exp.explicit_label = 'True'

In [ ]:
df_exp.columns = ['text','explicit']

In [ ]:
df_false = df[df['explicit'] == 'False']
df_true = df[df['explicit'] == 'True']
df_tomerge = df_false.loc[:10000]

In [ ]:
df=pd.concat([df_exp,df_tomerge,df_true],axis=0)

In [ ]:
df['text_len'] = df['text'].apply(lambda x: len(x.split(' ')))
df.head()

,text,explicit,text_len
0,"[Verse 1: Tyga] Ya'll know me, the still sam...",True,810
1,[B.G.] Niggas be hating 'cause B.G. got it ...,True,685
2,How the fuck are we supposed to know When I'...,True,327
3,Women they're good for nothing no maybe one th...,True,996
4,[Verse 1] I've been thinking 'bout you Yea...,True,435


In [ ]:
balance_counts = df.groupby('explicit')['explicit'].agg('count').values
balance_counts

array([4059, 2385])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=['Non-exp'],
    y=[balance_counts[0]],
    name='Non-exp',
    text=[balance_counts[0]],
    textposition='auto',
    marker_color=primary_blue
))
fig.add_trace(go.Bar(
    x=['exp'],
    y=[balance_counts[1]],
    name='exp',
    text=[balance_counts[1]],
    textposition='auto',
    marker_color=primary_grey
))
fig.update_layout(
    title='<span style="font-size:32px; font-family:Times New Roman">Dataset distribution by target</span>'
)
fig.show()

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
df['text_clean'] = df['text'].apply(clean_text)
df.head()

,text,explicit,text_len,text_clean
0,"[Verse 1: Tyga] Ya'll know me, the still sam...",True,810,yall know me the still same og young tyg ...
1,[B.G.] Niggas be hating 'cause B.G. got it ...,True,685,niggas be hating cause bg got it on top o...
2,How the fuck are we supposed to know When I'...,True,327,how the fuck are we supposed to know when im...
3,Women they're good for nothing no maybe one th...,True,996,women theyre good for nothing no maybe one thi...
4,[Verse 1] I've been thinking 'bout you Yea...,True,435,ive been thinking bout you yeah i been mi...


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = stopwords.words('english')

def remove_stopwords(text):
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text
    
df['text_clean'] = df['text_clean'].apply(remove_stopwords)
df.head()

,text,explicit,text_len,text_clean
0,"[Verse 1: Tyga] Ya'll know me, the still sam...",True,810,yall know still og young tyg hated niggas...
1,[B.G.] Niggas be hating 'cause B.G. got it ...,True,685,niggas hating cause bg got top line got r...
2,How the fuck are we supposed to know When I'...,True,327,fuck supposed know im monster way refuse d...
3,Women they're good for nothing no maybe one th...,True,996,women theyre good nothing maybe one thing se...
4,[Verse 1] I've been thinking 'bout you Yea...,True,435,ive thinking bout yeah missing hell oh ...


In [ ]:
stemmer = nltk.SnowballStemmer("english")
def preprocess_data(text):
    # Clean puntuation, urls, and so on
    text = clean_text(text)
    # Remove stopwords
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    # Stemm all the words in the sentence
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    
    return text

In [ ]:
df['text_clean'] = df['text_clean'].apply(preprocess_data)
df.head()

,text,explicit,text_len,text_clean
0,"[Verse 1: Tyga] Ya'll know me, the still sam...",True,810,yall know still og young tyg hate nigga ...
1,[B.G.] Niggas be hating 'cause B.G. got it ...,True,685,nigga hate caus bg got top line got ride ...
2,How the fuck are we supposed to know When I'...,True,327,fuck suppos know im monster way refus die ...
3,Women they're good for nothing no maybe one th...,True,996,women theyr good noth mayb one thing serv ne...
4,[Verse 1] I've been thinking 'bout you Yea...,True,435,ive think bout yeah miss hell oh need ...


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['explicit'])

df['target_encoded'] = le.transform(df['explicit'])
df.head()

,text,explicit,text_len,text_clean,target_encoded
0,"[Verse 1: Tyga] Ya'll know me, the still sam...",True,810,yall know still og young tyg hate nigga ...,1
1,[B.G.] Niggas be hating 'cause B.G. got it ...,True,685,nigga hate caus bg got top line got ride ...,1
2,How the fuck are we supposed to know When I'...,True,327,fuck suppos know im monster way refus die ...,1
3,Women they're good for nothing no maybe one th...,True,996,women theyr good noth mayb one thing serv ne...,1
4,[Verse 1] I've been thinking 'bout you Yea...,True,435,ive think bout yeah miss hell oh need ...,1


In [ ]:
x = df['text_clean']
y = df['target_encoded']

print(len(x), len(y))

6444 6444


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

4833 4833
1611 1611


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(x_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
x_train_dtm = vect.transform(x_train)
x_test_dtm = vect.transform(x_test)

In [ ]:
vect_tunned = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.1, max_df=0.7, max_features=200)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()

tfidf_transformer.fit(x_train_dtm)
x_train_tfidf = tfidf_transformer.transform(x_train_dtm)

x_train_tfidf

<4833x20387 sparse matrix of type '<class 'numpy.float64'>'
	with 392383 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

nb.fit(x_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
y_pred_class = nb.predict(x_test_dtm)
y_pred_prob = nb.predict_proba(x_test_dtm)[:, 1]

In [ ]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

conf_matrix(metrics.confusion_matrix(y_test, y_pred_class))

0.8646803227808815


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('bow', CountVectorizer()), 
                 ('tfid', TfidfTransformer()),  
                 ('model', MultinomialNB())])

In [ ]:
pipe.fit(x_train, y_train)

y_pred_class = pipe.predict(x_test)

print(metrics.accuracy_score(y_test, y_pred_class))

conf_matrix(metrics.confusion_matrix(y_test, y_pred_class))

0.8075729360645562


In [ ]:
import xgboost as xgb

pipe = Pipeline([
    ('bow', CountVectorizer()), 
    ('tfid', TfidfTransformer()),  
    ('model', xgb.XGBClassifier(
        learning_rate=0.1,
        max_depth=7,
        n_estimators=80,
        use_label_encoder=False,
        eval_metric='auc'
    ))
])

In [ ]:
pipe.fit(x_train, y_train)

y_pred_class = pipe.predict(x_test)
y_pred_train = pipe.predict(x_train)

print('Train: {}'.format(metrics.accuracy_score(y_train, y_pred_train)))
print('Test: {}'.format(metrics.accuracy_score(y_test, y_pred_class)))

conf_matrix(metrics.confusion_matrix(y_test, y_pred_class))

Train: 0.9710324849989654
Test: 0.9261328367473619


In [ ]:
import joblib as jbl

jbl.dump(pipe, 'model_xgb_jbl')

['model_xgb_jbl']

In [ ]:
xgb = jbl.load('model_xgb_jbl')

In [ ]:
xgb.predict(x_train)

array([0, 0, 0, ..., 1, 0, 1])

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-03-25 03:23:33--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-03-25 03:23:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-03-25 03:23:34--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

glove

In [ ]:
%%capture
!unzip glove*.zip

## WEG

In [ ]:
texts = df['text_clean']
target = df['target_encoded']

In [ ]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(texts)

vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

22663

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def embed(corpus): 
    return word_tokenizer.texts_to_sequences(corpus)

longest_train = max(texts, key=lambda sentence: len(word_tokenize(sentence)))
length_long_sentence = len(word_tokenize(longest_train))

train_padded_sentences = pad_sequences(
    embed(texts), 
    length_long_sentence, 
    padding='post'
)

train_padded_sentences

array([[ 200,    5,   61, ...,    0,    0,    0],
       [  28,  165,   18, ...,    0,    0,    0],
       [  21,  716,    5, ...,    0,    0,    0],
       ...,
       [ 396,  272,   35, ...,    0,    0,    0],
       [ 383,   55,   51, ...,    0,    0,    0],
       [2426, 1052,  266, ...,    0,    0,    0]], dtype=int32)

In [ ]:
embeddings_dictionary = dict()
embedding_dim = 100

# Load GloVe 100D embeddings
with open('glove.6B.50d.txt') as fp:
    for line in fp.readlines():
        records = line.split()
        word = records[0]
        vector_dimensions = np.asarray(records[1:], dtype='float32')
        embeddings_dictionary [word] = vector_dimensions


KeyboardInterrupt: ignored

In [ ]:
embedding_matrix = np.zeros((vocab_length, embedding_dim))

for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        
embedding_matrix

In [ ]:
import plotly.figure_factory as ff

x_axes = ['non-exp', 'exp']
y_axes =  ['exp', 'non-exp']

def conf_matrix(z, x=x_axes, y=y_axes):
    
    z = np.flip(z, 0)

    z_text = [[str(y) for y in x] for x in z]

    fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')
    fig.update_layout(title_text='<b>Confusion matrix</b>',
                      xaxis = dict(title='Predicted value'),
                      yaxis = dict(title='Real value')
                     )

    fig['data'][0]['showscale'] = True
    
    return fig

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

# Train the model
nb.fit(x_train_dtm, y_train)

In [ ]:
y_pred_class = nb.predict(x_test_dtm)
y_pred_prob = nb.predict_proba(x_test_dtm)[:, 1]

In [ ]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

conf_matrix(metrics.confusion_matrix(y_test, y_pred_class))

#LSTM

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    train_padded_sentences, 
    target, 
    test_size=0.25
)

In [ ]:
def glove_lstm():
    model = Sequential()
    
    model.add(Embedding(
        input_dim=embedding_matrix.shape[0], 
        output_dim=embedding_matrix.shape[1], 
        weights = [embedding_matrix], 
        input_length=length_long_sentence
    ))
    
    model.add(Bidirectional(LSTM(
        length_long_sentence, 
        return_sequences = True, 
        recurrent_dropout=0.2
    )))
    
    model.add(GlobalMaxPool1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(length_long_sentence, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(length_long_sentence, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

model = glove_lstm()
model.summary()

In [ ]:
model = glove_lstm()

checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor = 'val_loss', 
    verbose = 1, 
    save_best_only = True
)
reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss', 
    factor = 0.2, 
    verbose = 1, 
    patience = 5,                        
    min_lr = 0.001
)
history = model.fit(
    X_train, 
    y_train, 
    epochs = 7,
    batch_size = 32,
    validation_data = (X_test, y_test),
    verbose = 1,
    callbacks = [reduce_lr, checkpoint]
)

In [ ]:
!pip install transformers

In [ ]:
from transformers import (
    DistilBertTokenizerFast, 
    TFDistilBertForSequenceClassification
)
import tensorflow as tf

In [ ]:
texts = df['text_clean']
target = df['target_encoded']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    texts, 
    target, 
    test_size=0.25
)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# encode the training and validation data
train_encodings = tokenizer(X_train.tolist(), truncation = True, padding = True)
test_encodings = tokenizer(X_test.tolist(), truncation = True, padding = True)

train_set = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings), y_train.values)).shuffle(1000).batch(32).repeat()

test_set = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings), y_test.values)).shuffle(1000).batch(32)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
except:
    strategy = tf.distribute.get_strategy()
    
print('Number of replicas in sync: ', strategy.num_replicas_in_sync)

In [ ]:
with strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained(
      'distilbert-base-uncased', num_labels = 1)

    optimizer = tf.keras.optimizers.Adam(learning_rate = 4e-5)
    model.compile(
        optimizer = optimizer, 
        loss = model.compute_loss, 
        metrics = ['accuracy'],
    )